In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import sys
import warnings
warnings.filterwarnings("ignore")
import os
import joblib
sys.path.append(os.path.abspath('../src'))
from data_tokenizer import procesar_texto
import data_loader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, recall_score, precision_score

In [17]:
def division_datos(tfidf, mensajes=True):
    '''
    Carga y segmenta los datos en entrenamiento, validacion y prueba dado el tokenizador escogido.
    Guarda el scaler para normalizar datos en la prediccion.
    Argumentos:
        * tfidf: True para usar TF-IDF, False para usar CountVectorizer
        * mensajes: Por default es True. Indica si se desea imprimir un diagnostico de cantidad de filas y primeros registros de las bases finales

    Retorno:
        * x_train, x_test, x_val
        * y_train, y_test, y_val
    '''
    # Cargando datos de acuerdo a tokenizador seleccionado
    if tfidf:
        vectorizador = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/vectorizador_tfidf.pkl')
        x = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/tweets_tfidf.pkl')
    else:
        vectorizador = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/vectorizador_tf.pkl')
        x = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/tweets_tf.pkl')
    
    y = joblib.load('C:/Users/gerb2/Documents/DEEPLEARNING/taller2_tweets/Modelo_Sentimientos/models/labels.pkl')
    
    # Division en train, test y validacion
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

    if mensajes:
        print("Dimensiones de X completa:", x.shape)
        print("Dimensiones de X train:", x_train.shape)
        print("Dimensiones de X test:", x_test.shape)

        print("\nPrimeros registros X test:")
        print(x_test.toarray()[:5])

        print("\nPrimeras 5 etiquetas")
        print(y[:5])
    
    return x_train, x_test, x_val, y_train, y_test, y_val

In [6]:
# Definir parámetros del vocabulario y la secuencia
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 130

In [7]:
# Modelo RNN Básico

model = Sequential([
    Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=64, input_length=MAX_SEQUENCE_LENGTH),
    SimpleRNN(64, return_sequences=False),
    Dense(1, activation='sigmoid')  # para clasificación binaria
])

In [8]:
#Compilar

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [18]:
from sklearn.utils import class_weight
import numpy as np

# Llamada correcta a la función division_datos
x_train, x_test, x_val, y_train, y_test, y_val = division_datos(tfidf=True)

# Calcular pesos de clase
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))
print("Pesos de clase:", class_weights_dict)

Dimensiones de X completa: (31962, 10000)
Dimensiones de X train: (23012, 10000)
Dimensiones de X test: (6393, 10000)

Primeros registros X test:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Primeras 5 etiquetas
0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64
Pesos de clase: {np.int64(0): np.float64(0.5374626307922272), np.int64(1): np.float64(7.173316708229426)}


In [ ]:
#Entrenar con pesos de clase

history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    batch_size=32,
    class_weight=class_weights_dict  # esto balancea las clases
)

In [ ]:
Epoch 1/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 25s 36ms/step - accuracy: 0.5740 - loss: 0.6887 - val_accuracy: 0.5838 - val_loss: 0.6987
Epoch 2/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 23s 33ms/step - accuracy: 0.5679 - loss: 0.6883 - val_accuracy: 0.5512 - val_loss: 0.6903
Epoch 3/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - accuracy: 0.5685 - loss: 0.6832 - val_accuracy: 0.6453 - val_loss: 0.6920
Epoch 4/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 42s 34ms/step - accuracy: 0.5556 - loss: 0.7025 - val_accuracy: 0.6551 - val_loss: 0.6896
Epoch 5/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - accuracy: 0.5421 - loss: 0.6917 - val_accuracy: 0.6837 - val_loss: 0.6893
Epoch 6/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - accuracy: 0.5321 - loss: 0.6974 - val_accuracy: 0.5643 - val_loss: 0.6944
Epoch 7/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 23s 33ms/step - accuracy: 0.5275 - loss: 0.6836 - val_accuracy: 0.3760 - val_loss: 0.7004
Epoch 8/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.5320 - loss: 0.6802 - val_accuracy: 0.5448 - val_loss: 0.7028
Epoch 9/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - accuracy: 0.5288 - loss: 0.6992 - val_accuracy: 0.5977 - val_loss: 0.6917
Epoch 10/10
699/699 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - accuracy: 0.5492 - loss: 0.6942 - val_accuracy: 0.5360 - val_loss: 0.7064

In [ ]:
# Evaluar en el set de prueba
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss en test: {loss:.4f}")
print(f"Accuracy en test: {accuracy:.4f}")

In [ ]:
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6383 - loss: 0.6963
Loss en test: 0.6967
Accuracy en test: 0.6397

In [ ]:
#Ver métricas más completas (confusion matrix, precision, recall, F1)

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predicciones
y_pred = model.predict(x_test)
y_pred_classes = (y_pred > 0.5).astype("int32")

# Reporte
print(classification_report(y_test, y_pred_classes))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión")
plt.show()


In [ ]:
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
              precision    recall  f1-score   support

           0       0.94      0.66      0.77      5203
           1       0.09      0.43      0.14       390

    accuracy                           0.64      5593
   macro avg       0.51      0.54      0.46      5593
weighted avg       0.88      0.64      0.73      5593

Precisión: De los casos que el modelo predijo como clase 1, solo el 9% eran correctos → muchos falsos positivos.

Recall: El modelo solo detectó el 43% de los positivos reales → se le están escapando bastantes.

F1-score: Balance entre precisión y recall. Muy bajo (0.14) en la clase 1 → modelo no está aprendiendo bien a detectar la clase minoritaria.

3410: Verdaderos Negativos (TN) → el modelo acertó con clase 0.

1793: Falsos Positivos (FP) → el modelo predijo 1 pero era 0.

222: Falsos Negativos (FN) → el modelo predijo 0 pero era 1.

168: Verdaderos Positivos (TP) → el modelo acertó con clase 1.

In [ ]:
# Visualizar el entrenamiento

plt.plot(history.history['accuracy'], label='Accuracy entrenamiento')
plt.plot(history.history['val_accuracy'], label='Accuracy validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.title('Precisión durante el entrenamiento')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Loss entrenamiento')
plt.plot(history.history['val_loss'], label='Loss validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Pérdida durante el entrenamiento')
plt.legend()
plt.show()

Precisión durante el entrenamiento:

La precisión de entrenamiento (línea azul) es bastante estable entre 0.53 y 0.55. Eso puede indicar que el modelo está aprendiendo muy lentamente o que está limitado por su capacidad.

La precisión de validación (línea naranja) varía mucho y cae bruscamente en algunas épocas (por ejemplo en la época 6).

Esta oscilación tan fuerte sugiere alta varianza, lo cual puede ser señal de sobreajuste (overfitting) o que el modelo no está generalizando bien.

Pérdida durante el entrenamiento:

La pérdida de entrenamiento (línea azul) baja muy lentamente, lo que puede indicar que el modelo no está aprendiendo eficientemente.

La pérdida de validación es inestable y en general tiende a subir → lo que también es una señal de overfitting.